In [1]:
from __future__ import print_function

In [2]:
%%sh

whoami
ls -l /dev/uio* /dev/mem
cat /sys/class/uio/uio?/maps/map0/name
cat /sys/class/uio/uio?/name
ls -l /sys/class/uio/uio0/maps/map0

parallella
crw-r----- 1 root kmem      1, 1 Jan  1  1970 /dev/mem
crw-rw---- 1 root plugdev 247, 0 Jan  1  1970 /dev/uio0
crw-rw---- 1 root plugdev 247, 1 Jan  1  1970 /dev/uio1
crw-rw---- 1 root plugdev 247, 2 Jan  1  1970 /dev/uio2
/scratch_mem@3e000000
/amba_pl/my_mult@70020000
/amba_pl/dma@40400000
scratch_mem
my_mult
dma
total 0
-r--r--r-- 1 root root 4096 Jan 10 12:56 addr
-r--r--r-- 1 root root 4096 Jan 10 12:56 name
-r--r--r-- 1 root root 4096 Jan 10 12:56 offset
-r--r--r-- 1 root root 4096 Jan 10 12:56 size


In [3]:
import numpy as np
from axidma import AxiDMA


def to_bin(x):
    vv = [ (x>>(32-(i+1)*4))&0xF for i in range(8)]
    return '{0:04b}_{1:04b}|{2:04b}_{3:04b}|{4:04b}_{5:04b}|{6:04b}_{7:04b}'.format( *vv )

dma = AxiDMA('dma')
dma

UIO: dma (uio2) sz:0x10000 @0x40400000

In [4]:
print('\n'.join([to_bin(x) for x in [dma.mm2s.cr, dma.s2mm.cr, dma.mm2s.st,dma.s2mm.st]]))

0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0000|0000_0000|0000_0001
0000_0000|0000_0000|0000_0000|0000_0001


In [5]:
def mult_model(src):
    from numpy import ndarray
    
    ab = src.view('uint16')
    a = ab[0::2]
    b = ab[1::2]
    N = len(a)
    
    dst    = ndarray(N, dtype='uint32')
    dst[:] = a.astype('uint32')*b
    
    return dst.view('uint8')

def report_stats(dt, n_in, n_out, item_sz = 4):
    from numpy import r_
    bb = r_[n_in, n_out, n_in + n_out]
    mbb  = bb*1e-6
    bps  = bb/dt
    mbps = mbb/dt
    N    = n_in/item_sz

    print("Memory    : %g Mb(in) + %g Mb(out) = %g Mb(total)"%(mbb[0], mbb[1], mbb[2]) )
    print("Took      : %g ms"%(dt*1e+3))
    print("Per item  : %g us"%( (dt*1e+6)/N ))
    print("Items/s   : %.1f"  %(N/dt))
    print("Throughput: %g Mb(in) + %g Mb(out) = %g Mb/s"%(mbps[0], mbps[1], mbps[2]))

In [6]:
import uio
reload(uio)
mem_uio = uio.UIO('scratch_mem')

#SZ=(1<<23)-64
SZ=(1<<20)

data = mem_uio.as_ndarray()

src_data = data[:SZ]
dst_data = data[SZ:SZ*2]

src_data[:] = np.random.randint(0,256,SZ)
dst_data[:] = 0xFF
expect_data = mult_model(src_data)

print("SRC   :",src_data[:8],'...')
print("DST   :",dst_data[:8],'...')
print("Expect:",expect_data[:8],'...')

SRC   : [253 118  87  71 227  60 218  79] ...
DST   : [255 255 255 255 255 255 255 255] ...
Expect: [251 154  40  33  78 230 253  18] ...


In [7]:
import time

t0 = time.time()
dma.launch(mem_uio.phy_buf(src_data), mem_uio.phy_buf(dst_data), enable_interrupts=True)
rr = dma.wait()
t_done = time.time()

if rr:
    print('DMA Completed Succesfully')
else:
    print('DMA FAILED')


if (expect_data == dst_data).all():
    print('SUCCESS: Data received as expected')
else:
    print('FAILED: dst and src do not match up')

print("SRC   :",src_data[:8],'...')
print("DST   :",dst_data[:8],'...')
print("Expect:", expect_data[:8],'...')

print('\nTime Stats')
print('='*80)
report_stats(t_done - t0, SZ, SZ)


DMA Completed Succesfully
SUCCESS: Data received as expected
SRC   : [253 118  87  71 227  60 218  79] ...
DST   : [251 154  40  33  78 230 253  18] ...
Expect: [251 154  40  33  78 230 253  18] ...

Time Stats
Memory    : 1.04858 Mb(in) + 1.04858 Mb(out) = 2.09715 Mb(total)
Took      : 6.19292 ms
Per item  : 0.0236241 us
Items/s   : 42329610.3
Throughput: 169.318 Mb(in) + 169.318 Mb(out) = 338.637 Mb/s


In [8]:
#!dtc -I fs -O dts /proc/device-tree/ 